In [ ]:
# Import necessary libraries
import nest  # NEST simulator for spiking neural networks
import matplotlib.pyplot as plt  # For plotting results
import ipywidgets as widgets  # For interactive sliders
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider

# Set NEST verbosity to suppress extra NEST logs
nest.set_verbosity("M_WARNING")

   Simulates two neurons:
    - Neuron 1: Receives current input only.
    - Neuron 2 gets input only via synaptic connection from Neuron 1 (Conn: Neuron 1 > Neuron 2).

In [ ]:
def run_two_neurons(I_e, sim_time, weight_between):
    # Reset the NEST kernel (to clear previous simulations)
    nest.ResetKernel()

    # Set the simulation resolution
    nest.SetKernelStatus({"resolution": 0.01})

    # Create two IAF neurons
    neuron1 = nest.Create('iaf_psc_alpha')
    neuron2 = nest.Create('iaf_psc_alpha')

    # Set neuron parameters
    nest.SetStatus(neuron1, {
        "I_e": I_e,
        "C_m": 250,
        "tau_m": 20,
        "V_reset": -70,
        "V_th": -55,
        "t_ref": 8
    })
    nest.SetStatus(neuron2, {
        "C_m": 250,
        "tau_m": 20,
        "V_reset": -70,
        "V_th": -55,
        "t_ref": 8
    })

    # Create devices for recording and stimulation
    spike_recorder1 = nest.Create('spike_recorder')
    spike_recorder2 = nest.Create('spike_recorder')
    multimeter1 = nest.Create('multimeter')
    multimeter2 = nest.Create('multimeter')

    # Configure the multimeters
    nest.SetStatus(multimeter1, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter2, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter1, {"interval": 0.01})
    nest.SetStatus(multimeter2, {"interval": 0.01})


    # Connect devices
    nest.Connect(multimeter1, neuron1)
    nest.Connect(multimeter2, neuron2)
    nest.Connect(neuron1, spike_recorder1)
    nest.Connect(neuron2, spike_recorder2)

    # Connections between neurons
    nest.Connect(neuron1, neuron2, syn_spec={"weight": weight_between})

    # Simulate for the specified time
    nest.Simulate(sim_time)

    # Retrieve and plot data for neuron 1
    events1 = nest.GetStatus(multimeter1, 'events')[0]
    time1 = events1['times']
    V_m1 = events1['V_m']
    spikes1 = nest.GetStatus(spike_recorder1, 'events')[0]
    spike_times1 = spikes1['times']

    # Retrieve and plot data for neuron 2
    events2 = nest.GetStatus(multimeter2, 'events')[0]
    time2 = events2['times']
    V_m2 = events2['V_m']
    spikes2 = nest.GetStatus(spike_recorder2, 'events')[0]
    spike_times2 = spikes2['times']

    # Print spike times
    print(f'Neuron 1: Spike Times: {", ".join(map(str, spike_times1))}')
    print(f'Neuron 2: Spike Times: {", ".join(map(str, spike_times2))}')

    # Calculate firing rates
    firing_rate1 = len(spike_times1) / (sim_time / 1000.0)
    firing_rate2 = len(spike_times2) / (sim_time / 1000.0)
    print(f'Neuron 1: Firing Rate: {firing_rate1:.2f} Hz')
    print(f'Neuron 2: Firing Rate: {firing_rate2:.2f} Hz')

    # Plot the membrane potentials with detailed markers
    plt.figure(figsize=(12, 6))
    plt.plot(time1, V_m1, label='Neuron 1 Membrane Potential (V_m)', linewidth=1.5)
    plt.plot(time2, V_m2, label='Neuron 2 Membrane Potential (V_m)', linewidth=1.5)

    # Add threshold lines for clarity

    # Labels and grid
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.title('IAF Neurons Membrane Potential Over Time')
    plt.axhline(-55, color='gray', linestyle='--', label='Threshold (V_th)')
    plt.legend()
    plt.grid()
    plt.show()



# Interactive widgets for input parameters
I_e = widgets.FloatSlider(value=416.0, min=0.0, max=100000.0, step=0.01, description='I_e (pA):')
sim_time = widgets.FloatSlider(value=100.0, min=1.0, max=60000.0, step=1.0, description='Sim Time (ms):')
weight_between = widgets.FloatSlider(value=953.30, min=-10.0, max=100000.0, step=0.1, description='Weight Between:')

widgets.interact(run_two_neurons, 
I_e=I_e, sim_time=sim_time, weight_between=weight_between
)


Simulates two neurons:
    - Neuron 1: Receives current input only and is self-connected (Neuron 1 > Neuron 1).
    - Neuron 2 gets input only via synaptic connection from Neuron 1 (Neuron 1 > Neuron 2).

In [ ]:
def run_two_neurons_one_self(I_e, sim_time, weight_self, weight_between):
    # Reset the NEST kernel (to clear previous simulations)
    nest.ResetKernel()

    # Set the simulation resolution
    nest.SetKernelStatus({"resolution": 0.01})
    
    # Create two IAF neurons
    neuron1 = nest.Create('iaf_psc_alpha')
    neuron2 = nest.Create('iaf_psc_alpha')

    # Set neuron parameters
    nest.SetStatus(neuron1, {
        "I_e": I_e,
        "t_ref": 8,
        "C_m": 250,
        "tau_m": 20,
        "V_reset": -70,
        "V_th": -55
    })
    nest.SetStatus(neuron2, {
        "t_ref": 8,
        "C_m": 250,
        "tau_m": 20,
        "V_reset": -70,
        "V_th": -55
    })

    # Create devices for recording and stimulation
    spike_recorder1 = nest.Create('spike_recorder')
    spike_recorder2 = nest.Create('spike_recorder')
    multimeter1 = nest.Create('multimeter')
    multimeter2 = nest.Create('multimeter')
    nest.SetStatus(multimeter1, {"interval": 0.01})
    nest.SetStatus(multimeter2, {"interval": 0.01})
    
    # Configure the multimeters
    nest.SetStatus(multimeter1, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter2, {"record_from": ["V_m"]})

    # Connect devices
    nest.Connect(multimeter1, neuron1)
    nest.Connect(multimeter2, neuron2)
    nest.Connect(neuron1, spike_recorder1)
    nest.Connect(neuron2, spike_recorder2)

    # Connections between neurons
    nest.Connect(neuron1, neuron1, syn_spec={"weight": weight_self})
    nest.Connect(neuron1, neuron2, syn_spec={"weight": weight_between})

    # Simulate for the specified time
    nest.Simulate(sim_time)

    # Retrieve and plot data for neuron 1
    events1 = nest.GetStatus(multimeter1, 'events')[0]
    time1 = events1['times']
    V_m1 = events1['V_m']
    spikes1 = nest.GetStatus(spike_recorder1, 'events')[0]
    spike_times1 = spikes1['times']

    # Retrieve and plot data for neuron 2
    events2 = nest.GetStatus(multimeter2, 'events')[0]
    time2 = events2['times']
    V_m2 = events2['V_m']
    spikes2 = nest.GetStatus(spike_recorder2, 'events')[0]
    spike_times2 = spikes2['times']

    # Print spike times
    print(f'Neuron 1 Spike Times: {", ".join(map(str, spike_times1))}')
    print(f'Neuron 2 Spike Times: {", ".join(map(str, spike_times2))}')

    # Calculate firing rates
    firing_rate1 = len(spike_times1) / (sim_time / 1000.0)
    firing_rate2 = len(spike_times2) / (sim_time / 1000.0)
    print(f'Neuron 1 Firing Rate: {firing_rate1:.2f} Hz')
    print(f'Neuron 2 Firing Rate: {firing_rate2:.2f} Hz')

    # Plot the membrane potentials with detailed markers
    plt.figure(figsize=(12, 6))
    plt.plot(time1, V_m1, label='Neuron 1 Membrane Potential (V_m)', linewidth=1.5)
    plt.plot(time2, V_m2, label='Neuron 2 Membrane Potential (V_m)', linewidth=1.5)

    # Add spike markers
    plt.axhline(-55, color='gray', linestyle='--', label='Threshold (V_th)')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.title('IAF Neurons Membrane Potential Over Time')
    plt.axhline(-55, color='gray', linestyle='--', label='Threshold (V_th)')
    plt.legend()
    plt.grid()
    plt.show()

# Interactive widgets for input parameters
I_e = widgets.FloatSlider(value=187.52, min=0.0, max=100000.0, step=0.01, description='I_e (pA):')
sim_time = widgets.FloatSlider(value=1000.0, min=1.0, max=60000.0, step=1.0, description='Sim Time (ms):')
weight_self = widgets.FloatSlider(value=4511.0, min=-10.0, max=100000.0, step=0.01, description='Weight Self:')
weight_between = widgets.FloatSlider(value=953.0, min=-10.0, max=100000.0, step=0.01, description='Weight Between:')

widgets.interact(run_two_neurons_one_self, 
I_e=I_e, sim_time=sim_time, weight_self=weight_self, weight_between=weight_between
)


Simulates two neurons:
    - Neuron 1: Receives current input only.
    - Both neurons are self-connected and connected birectional to each other (1 > 1, 2 > 2, 1 <-> 2).

In [ ]:
def two_neurons_network(I_e, t_ref, sim_time, weight_self, weight_between):
    # Reset the NEST kernel (to clear previous simulations)
    nest.ResetKernel()

    # Set the simulation resolution
    nest.SetKernelStatus({"resolution": 0.001})

    # Create two IAF neurons
    neuron1 = nest.Create('iaf_psc_alpha')
    neuron2 = nest.Create('iaf_psc_alpha')

    # Set neuron parameters
    nest.SetStatus(neuron1, {
        "I_e": I_e,
        "C_m": 250,
        "tau_m": 20,
        "V_reset": -70,
        "V_th": -55,
        "t_ref": t_ref
    })
    nest.SetStatus(neuron2, {
        "C_m": 250,
        "tau_m": 20,
        "V_reset": -70,
        "V_th": -55,
        "t_ref": t_ref
    })

    # Create devices for recording and stimulation
    spike_recorder1 = nest.Create('spike_recorder')
    spike_recorder2 = nest.Create('spike_recorder')
    multimeter1 = nest.Create('multimeter')
    multimeter2 = nest.Create('multimeter')

    # Configure the multimeters
    nest.SetStatus(multimeter1, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter2, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter1, {"interval": 0.001})
    nest.SetStatus(multimeter2, {"interval": 0.001})
    
    # Connect devices
    nest.Connect(multimeter1, neuron1)
    nest.Connect(multimeter2, neuron2)
    nest.Connect(neuron1, spike_recorder1)
    nest.Connect(neuron2, spike_recorder2)

    # Self-connections
    nest.Connect(neuron1, neuron1, syn_spec={"weight": weight_self})
    nest.Connect(neuron2, neuron2, syn_spec={"weight": weight_self})

    # Connections between neurons
    nest.Connect(neuron1, neuron2, syn_spec={"weight": weight_between})
    nest.Connect(neuron2, neuron1, syn_spec={"weight": weight_between})

    # Simulate for the specified time
    nest.Simulate(sim_time)

    # Retrieve and plot data for neuron 1
    events1 = nest.GetStatus(multimeter1, 'events')[0]
    time1 = events1['times']
    V_m1 = events1['V_m']
    spikes1 = nest.GetStatus(spike_recorder1, 'events')[0]
    spike_times1 = spikes1['times']

    # Retrieve and plot data for neuron 2
    events2 = nest.GetStatus(multimeter2, 'events')[0]
    time2 = events2['times']
    V_m2 = events2['V_m']
    spikes2 = nest.GetStatus(spike_recorder2, 'events')[0]
    spike_times2 = spikes2['times']

    # Print spike times
    print(f'Neuron 1 Spike Times: {", ".join(map(str, spike_times1))}')
    print(f'Neuron 2 Spike Times: {", ".join(map(str, spike_times2))}')

    # Calculate firing rates
    firing_rate1 = len(spike_times1) / (sim_time / 1000.0)
    firing_rate2 = len(spike_times2) / (sim_time / 1000.0)
    print(f'Neuron 1 Firing Rate: {firing_rate1:.2f} Hz')
    print(f'Neuron 2 Firing Rate: {firing_rate2:.2f} Hz')

    # Plot the membrane potentials
    plt.figure(figsize=(10, 6))
    plt.plot(time1, V_m1, label='Neuron 1 Membrane Potential (V_m)')
    plt.plot(time2, V_m2, label='Neuron 2 Membrane Potential (V_m)')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.title('IAF Neurons Membrane Potential Over Time')
    plt.axhline(-55, color='gray', linestyle='--', label='Threshold (V_th)')
    plt.legend()
    plt.grid()
    plt.show()

# Interactive widgets for input parameters
I_e = widgets.FloatSlider(value=187.52, min=0.0, max=100000.0, step=0.01, description='I_e (pA):')
sim_time = widgets.FloatSlider(value=1000.0, min=1.0, max=60000.0, step=1.0, description='Sim Time (ms):')
t_ref = widgets.FloatSlider(value=8.0, min=0.0, max=100.0, step=1.0, description='t_ref (ms):')
weight_self = widgets.FloatSlider(value=4511.0, min=-1000.0, max=100000.0, step=0.1, description='Weight Self:')
weight_between = widgets.FloatSlider(value=558.0, min=-10.0, max=100000.0, step=0.01, description='Weight Between:')

widgets.interact(two_neurons_network, 
I_e=I_e, t_ref=t_ref, sim_time=sim_time, weight_self=weight_self, weight_between=weight_between
)
